<a href="https://colab.research.google.com/github/ananya1331/SER-notebooks/blob/main/ECAPA_TDNN_Demo_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q yt-dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 44.7 MB/s eta 0:00:00


In [ ]:
import os
import torch
import torchaudio
import numpy as np
import librosa
import soundfile as sf
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from typing import List, Dict, Tuple
from IPython.display import Audio, display, HTML
import warnings
warnings.filterwarnings('ignore')

print("✓ All imports successful!\n")

# ============================================================
# YOUTUBE AUDIO DOWNLOAD
# ============================================================

def download_youtube_audio(url: str, output_path: str = "./audio") -> str:
    """Download audio from YouTube URL"""
    import yt_dlp
    os.makedirs(output_path, exist_ok=True)

    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{'key': 'FFmpegExtractAudio', 'preferredcodec': 'wav'}],
        'outtmpl': os.path.join(output_path, '%(title)s.%(ext)s'),
        'quiet': True,
        'no_warnings': True,
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=True)
        filename = ydl.prepare_filename(info)
        audio_file = filename.rsplit('.', 1)[0] + '.wav'
    return audio_file

# ============================================================
# SPEAKER VERIFICATION WITH ECAPA-TDNN
# ============================================================

class SpeakerVerificationSystem:
    """Speaker Verification using ECAPA-TDNN from SpeechBrain"""

    def __init__(self):
        from speechbrain.pretrained import EncoderClassifier
        print("🔄 Loading ECAPA-TDNN model from SpeechBrain...")
        self.classifier = EncoderClassifier.from_hparams(
            source="speechbrain/spkrec-ecapa-voxceleb",
            savedir="pretrained_models/spkrec-ecapa-voxceleb"
        )
        self.speaker_name = None
        print("✅ Model loaded successfully!\n")

    def extract_embedding(self, audio_path: str) -> np.ndarray:
        """Extract speaker embedding from audio file"""
        print(f"   🔍 Extracting embedding from: {Path(audio_path).name}")
        signal, sr = torchaudio.load(audio_path)
        if sr != 16000:
            print(f"   🔄 Resampling from {sr}Hz to 16000Hz...")
            signal = torchaudio.transforms.Resample(sr, 16000)(signal)
        with torch.no_grad():
            embedding = self.classifier.encode_batch(signal)
        print(f"   ✅ Embedding extracted (shape: {embedding.shape})")
        return embedding.squeeze().cpu().numpy()

    def enroll_speaker(self, audio_files: List[str], speaker_name: str):
        """Enroll a speaker using multiple audio samples"""
        print(f"\n{'='*60}")
        print(f"👤 ENROLLING SPEAKER: {speaker_name}")
        print(f"{'='*60}")
        self.speaker_name = speaker_name
        embeddings = []

        for i, audio_file in enumerate(audio_files):
            print(f"\n📝 Processing enrollment sample {i+1}/{len(audio_files)}:")
            embedding = self.extract_embedding(audio_file)
            embeddings.append(embedding)

        print(f"\n🧮 Computing average enrollment embedding...")
        self.enrollment_embedding = np.mean(embeddings, axis=0)
        print(f"✅ Enrollment complete! Created voice profile from {len(audio_files)} samples")
        print(f"{'='*60}\n")

    def verify_speaker(self, test_audio: str, threshold: float = 0.25) -> Dict:
        """Verify if test audio matches enrolled speaker"""
        print(f"   🔍 Verifying: {Path(test_audio).name}")
        test_emb = self.extract_embedding(test_audio)

        print(f"   📊 Computing similarity score...")
        similarity = cosine_similarity(
            self.enrollment_embedding.reshape(1, -1),
            test_emb.reshape(1, -1)
        )[0][0]

        score = float(similarity * 100)
        is_verified = similarity > threshold
        confidence = 'HIGH' if abs(similarity - threshold) > 0.15 else 'MEDIUM' if abs(similarity - threshold) > 0.08 else 'LOW'

        status = "✅ VERIFIED" if is_verified else "❌ NOT VERIFIED"
        print(f"   {status} | Score: {score:.2f} | Confidence: {confidence}\n")

        return {
            'speaker_name': self.speaker_name,
            'similarity': float(similarity),
            'score': score,
            'verified': is_verified,
            'threshold': threshold,
            'confidence': confidence
        }

# ============================================================
# AUDIO SEGMENTATION
# ============================================================

def split_audio_into_segments(audio_file: str, segment_duration: int = 10, output_dir: str = "./segments") -> List[str]:
    """Split audio into segments for enrollment and testing"""
    print(f"   ✂️  Splitting audio into {segment_duration}-second segments...")
    os.makedirs(output_dir, exist_ok=True)
    audio, sr = librosa.load(audio_file, sr=16000)
    segment_length = segment_duration * sr

    segments = []
    for i in range(0, len(audio), segment_length):
        segment = audio[i:i+segment_length]
        if len(segment) >= sr * 3:  # At least 3 seconds
            segment_file = os.path.join(output_dir, f"segment_{i//segment_length:03d}.wav")
            sf.write(segment_file, segment, sr)
            segments.append(segment_file)

    print(f"   ✅ Created {len(segments)} segments")
    return segments

# ============================================================
# AUDIO PLAYBACK FUNCTIONS
# ============================================================

def play_enrollment_samples(audio_files: List[str]):
    """Play enrollment samples"""
    print("\n" + "="*60)
    print("🎤 ENROLLMENT SAMPLES - Voice Profile Being Created")
    print("="*60)
    print("Listen to these samples to hear the speaker being enrolled:\n")

    for i, audio_file in enumerate(audio_files):
        print(f"📢 Sample {i+1}/{len(audio_files)}: {Path(audio_file).name}")
        display(Audio(audio_file, autoplay=False, rate=16000))
        print()
    print("="*60)

def play_test_samples_with_results(test_segments: List[str], results: List[Dict]):
    """Play test segments with their verification results"""
    print("\n" + "="*60)
    print("🎯 VERIFICATION RESULTS - Test Each Audio Segment")
    print("="*60)
    print("Listen to each segment and see if it matches the enrolled speaker:\n")

    for i, (audio_file, result) in enumerate(zip(test_segments, results)):
        verified_emoji = "✅" if result['verified'] else "❌"
        color = "#d4edda" if result['verified'] else "#f8d7da"
        border_color = "#28a745" if result['verified'] else "#dc3545"

        # Display result box
        display(HTML(f"""
        <div style='padding:15px; background-color:{color}; border-left:5px solid {border_color};
                    margin:15px 0; border-radius:5px; font-family:Arial;'>
            <h3 style='margin:0 0 10px 0;'>{verified_emoji} Test Segment {i+1}/{len(test_segments)}</h3>
            <p style='margin:5px 0;'><strong>Score:</strong> {result['score']:.2f}/100</p>
            <p style='margin:5px 0;'><strong>Similarity:</strong> {result['similarity']:.4f}</p>
            <p style='margin:5px 0;'><strong>Status:</strong> {'VERIFIED ✓' if result['verified'] else 'NOT VERIFIED ✗'}</p>
            <p style='margin:5px 0;'><strong>Confidence:</strong> {result['confidence']}</p>
        </div>
        """))

        # Audio player
        print(f"🔊 Play audio segment {i+1}:")
        display(Audio(audio_file, autoplay=False, rate=16000))
        print("\n" + "-"*60 + "\n")

    print("="*60)

# ============================================================
# VISUALIZATION
# ============================================================

def visualize_verification_results(results: List[Dict]):
    """Visualize speaker verification results"""
    print("\n📊 Generating comprehensive visualization...")

    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('🎯 Speaker Verification Analysis - ECAPA-TDNN', fontsize=18, fontweight='bold')

    scores = [r['score'] for r in results]
    similarities = [r['similarity'] for r in results]
    verified = [r['verified'] for r in results]
    segments = [f"Seg {i+1}" for i in range(len(results))]

    # Plot 1: Verification Scores
    colors = ['#28a745' if v else '#dc3545' for v in verified]
    bars = axes[0, 0].bar(segments, scores, color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)
    axes[0, 0].axhline(y=results[0]['threshold']*100, color='orange', linestyle='--', linewidth=2, label=f'Threshold ({results[0]["threshold"]*100:.0f})')
    axes[0, 0].set_title('Verification Scores by Segment', fontsize=14, fontweight='bold')
    axes[0, 0].set_ylabel('Score (0-100)', fontsize=12)
    axes[0, 0].set_xlabel('Audio Segment', fontsize=12)
    axes[0, 0].legend(fontsize=10)
    axes[0, 0].grid(axis='y', alpha=0.3)
    axes[0, 0].tick_params(axis='x', rotation=45)

    # Add value labels on bars
    for bar in bars:
        height = bar.get_height()
        axes[0, 0].text(bar.get_x() + bar.get_width()/2., height,
                       f'{height:.1f}', ha='center', va='bottom', fontsize=9)

    # Plot 2: Similarity Distribution
    axes[0, 1].hist(similarities, bins=15, color='skyblue', edgecolor='black', alpha=0.8, linewidth=1.5)
    axes[0, 1].axvline(x=results[0]['threshold'], color='orange', linestyle='--', linewidth=2, label='Threshold')
    axes[0, 1].set_title('Similarity Score Distribution', fontsize=14, fontweight='bold')
    axes[0, 1].set_xlabel('Cosine Similarity', fontsize=12)
    axes[0, 1].set_ylabel('Frequency', fontsize=12)
    axes[0, 1].legend(fontsize=10)
    axes[0, 1].grid(axis='y', alpha=0.3)

    # Plot 3: Verification Rate Pie Chart
    verified_count = sum(verified)
    failed_count = len(verified) - verified_count
    colors_pie = ['#28a745', '#dc3545']
    explode = (0.05, 0.05)

    wedges, texts, autotexts = axes[1, 0].pie(
        [verified_count, failed_count],
        labels=['Verified', 'Failed'],
        colors=colors_pie,
        autopct='%1.1f%%',
        startangle=90,
        explode=explode,
        shadow=True,
        textprops={'fontsize': 12, 'fontweight': 'bold'}
    )
    axes[1, 0].set_title(f'Verification Rate\n✅ {verified_count}/{len(verified)} Verified',
                         fontsize=14, fontweight='bold')

    # Plot 4: Confidence Distribution
    conf_counts = {'HIGH': 0, 'MEDIUM': 0, 'LOW': 0}
    for r in results:
        conf_counts[r['confidence']] += 1

    conf_colors = {'HIGH': '#28a745', 'MEDIUM': '#ffc107', 'LOW': '#dc3545'}
    bars2 = axes[1, 1].bar(
        conf_counts.keys(),
        conf_counts.values(),
        color=[conf_colors[k] for k in conf_counts.keys()],
        alpha=0.8,
        edgecolor='black',
        linewidth=1.5
    )
    axes[1, 1].set_title('Confidence Level Distribution', fontsize=14, fontweight='bold')
    axes[1, 1].set_ylabel('Count', fontsize=12)
    axes[1, 1].set_xlabel('Confidence Level', fontsize=12)
    axes[1, 1].grid(axis='y', alpha=0.3)

    # Add value labels
    for bar in bars2:
        height = bar.get_height()
        axes[1, 1].text(bar.get_x() + bar.get_width()/2., height,
                       f'{int(height)}', ha='center', va='bottom', fontsize=11, fontweight='bold')

    plt.tight_layout()
    plt.show()

    # Print detailed summary
    print("\n" + "="*60)
    print("📋 FINAL VERIFICATION SUMMARY")
    print("="*60)
    print(f"🎤 Speaker: {results[0]['speaker_name']}")
    print(f"📊 Total Segments Tested: {len(results)}")
    print(f"✅ Verified: {verified_count} ({verified_count/len(results)*100:.1f}%)")
    print(f"❌ Failed: {failed_count} ({failed_count/len(results)*100:.1f}%)")
    print(f"📈 Average Score: {np.mean(scores):.2f}/100")
    print(f"📉 Score Range: {min(scores):.2f} - {max(scores):.2f}")
    print(f"🎯 Average Similarity: {np.mean(similarities):.4f}")
    print(f"🔒 Threshold: {results[0]['threshold']:.4f}")
    print(f"💪 High Confidence: {conf_counts['HIGH']}")
    print(f"⚠️  Medium Confidence: {conf_counts['MEDIUM']}")
    print(f"⚠️  Low Confidence: {conf_counts['LOW']}")
    print("="*60)

# ============================================================
# MAIN DEMO EXECUTION
# ============================================================

def run_speaker_verification_demo(youtube_urls: List[str],
                                   speaker_name: str = "Ranveer Allahbadia",
                                   num_enrollment: int = 3,
                                   play_audio: bool = True):
    """
    Main demo function for speaker verification with detailed progress

    Args:
        youtube_urls: List of YouTube URLs
        speaker_name: Name of the speaker to verify
        num_enrollment: Number of samples to use for enrollment
        play_audio: Whether to play audio samples (True for demo)
    """

    print("\n" + "="*60)
    print("🎬 ECAPA-TDNN SPEAKER VERIFICATION DEMO")
    print("="*60)
    print(f"🎯 Target Speaker: {speaker_name}")
    print(f"📹 Videos to Process: {len(youtube_urls[:6])}")
    print(f"🎤 Enrollment Samples: {num_enrollment}")
    print(f"🔊 Audio Playback: {'Enabled' if play_audio else 'Disabled'}")
    print("="*60)

    # Step 1: Download audio from YouTube
    print("\n📥 STEP 1: Downloading Audio from YouTube")
    print("-"*60)
    audio_files = []
    for i, url in enumerate(youtube_urls[:6]):
        print(f"\n🎥 Downloading video {i+1}/{min(len(youtube_urls), 6)}...")
        print(f"   URL: {url}")
        try:
            audio_file = download_youtube_audio(url)
            audio_files.append(audio_file)
            print(f"   ✅ Downloaded: {Path(audio_file).name}")
            print(f"   📁 Size: {os.path.getsize(audio_file) / (1024*1024):.2f} MB")
        except Exception as e:
            print(f"   ❌ Error: {e}")

    print(f"\n✅ Successfully downloaded {len(audio_files)} audio files")

    if len(audio_files) < 2:
        print("\n❌ ERROR: Need at least 2 audio files. Please check URLs.")
        return None, None

    # Step 2: Segment audio files
    print("\n✂️  STEP 2: Segmenting Audio Files")
    print("-"*60)
    all_segments = []
    for i, audio_file in enumerate(audio_files):
        print(f"\n📁 Processing file {i+1}/{len(audio_files)}: {Path(audio_file).name}")
        segments = split_audio_into_segments(audio_file, segment_duration=10)
        all_segments.extend(segments)

    print(f"\n✅ Total segments created: {len(all_segments)}")
    print(f"   Each segment is 10 seconds long")

    # Step 3: Show enrollment samples
    enrollment_segments = all_segments[:num_enrollment]

    if play_audio:
        play_enrollment_samples(enrollment_segments)
        input("\n⏸️  Press ENTER after listening to enrollment samples to continue...")

    # Step 4: Initialize and enroll speaker
    print("\n🤖 STEP 3: Initializing ECAPA-TDNN Model & Enrolling Speaker")
    print("-"*60)
    verifier = SpeakerVerificationSystem()
    verifier.enroll_speaker(enrollment_segments, speaker_name)

    # Step 5: Test verification
    print("\n🔍 STEP 4: Testing Speaker Verification")
    print("-"*60)
    test_segments = all_segments[num_enrollment:num_enrollment+10]
    print(f"Testing {len(test_segments)} segments...\n")

    results = []
    for i, seg in enumerate(test_segments):
        print(f"🎯 Test {i+1}/{len(test_segments)}:")
        result = verifier.verify_speaker(seg, threshold=0.25)
        results.append(result)

    print(f"\n✅ Verification complete for {len(results)} segments")

    # Step 6: Show test samples with results
    if play_audio:
        play_test_samples_with_results(test_segments, results)
        input("\n⏸️  Press ENTER to see final visualization and summary...")

    # Step 7: Visualize results
    print("\n📊 STEP 5: Generating Final Visualization")
    print("-"*60)
    visualize_verification_results(results)

    print("\n✅ Demo Complete! Results saved in verifier and results variables.")

    return verifier, results

# ============================================================
# YOUTUBE URLS - READY TO USE
# ============================================================

YOUTUBE_URLS = [
    "https://youtu.be/e-sycEw4gyc",  # with Janhvi Kapoor
    "https://youtu.be/OcISVEh1jyw",  # with Priyanka Chopra
    "https://youtu.be/8kS99Dqj6us",  # with Kajol
    "https://youtu.be/Mwpsr1iHStY",  # with Kriti Sanon
    "https://youtu.be/q3k8Ax8gBAI",  # with KSI
    "https://youtu.be/M7x6A9hMeUo",  # with Sourabh Raaj
]

print("\n" + "="*60)
print("✅ CODE LOADED AND READY!")
print("="*60)
print("\n🎯 To run the INTERACTIVE DEMO with audio playback:")
print(">>> verifier, results = run_speaker_verification_demo(YOUTUBE_URLS, play_audio=True)")
print("\n⚡ To run WITHOUT audio (faster):")
print(">>> verifier, results = run_speaker_verification_demo(YOUTUBE_URLS, play_audio=False)")
print("="*60)

✓ All imports successful!


✅ CODE LOADED AND READY!

🎯 To run the INTERACTIVE DEMO with audio playback:
>>> verifier, results = run_speaker_verification_demo(YOUTUBE_URLS, play_audio=True)

⚡ To run WITHOUT audio (faster):
>>> verifier, results = run_speaker_verification_demo(YOUTUBE_URLS, play_audio=False)


In [ ]:
verifier, results = run_speaker_verification_demo(YOUTUBE_URLS, play_audio=True)


🎬 ECAPA-TDNN SPEAKER VERIFICATION DEMO
🎯 Target Speaker: Ranveer Allahbadia
📹 Videos to Process: 6
🎤 Enrollment Samples: 3
🔊 Audio Playback: Enabled

📥 STEP 1: Downloading Audio from YouTube
------------------------------------------------------------

🎥 Downloading video 1/6...
   URL: https://youtu.be/e-sycEw4gyc
   ✅ Downloaded: Janhvi Kapoor Like Never Before - On Love, Family, Heartbreaks & Healing ｜ TRS.wav
   📁 Size: 727.14 MB

🎥 Downloading video 2/6...
   URL: https://youtu.be/OcISVEh1jyw
   ✅ Downloaded: Priyanka Chopra on Self Confidence, Entrepreneurship, Family & Success ｜ The Ranveer Show 256.wav
   📁 Size: 559.73 MB

🎥 Downloading video 3/6...
   URL: https://youtu.be/8kS99Dqj6us
   ✅ Downloaded: Actor Kajol - Funny & Unfiltered Like Never Before ｜ The Ranveer Show 260.wav
   📁 Size: 694.40 MB

🎥 Downloading video 4/6...
   URL: https://youtu.be/Mwpsr1iHStY
   ✅ Downloaded: Kriti Sanon Ki Unfiltered Kahani - Bollywood, Dosti & Goals, Money, Inner Peace ｜ TRS.wav
   📁 Si


📢 Sample 2/3: segment_001.wav



📢 Sample 3/3: segment_002.wav


KeyboardInterrupt: Interrupted by user